In [7]:
import numpy as np
import pandas as pd
import random

In [8]:
class Node():
    def __init__(self, data=None, left=None, right=None):
        self.data = data 
        self.left = left
        self.right = right



In [9]:
class geneNode(Node):
    def __init__(self, minDepth=1, maxDepth=2, functions=[]):
        Node.__init__(self)
        self.minDepth = minDepth
        self.maxDepth = maxDepth
        self.functions = functions


    def buildRandomTree(self, d=0):
        
        def insertRandomNode(node, depth=0):
            
            if depth == self.maxDepth:
                node.data = random.randint(-1, 1)

            elif depth == 0:
                node.data = self.functions[random.randint(0, len(self.functions)-1)]

            else:            
                if random.random() > 0.5:
                    node.data = random.randint(-1, 1)
                else:
                    node.data = self.functions[random.randint(0, len(self.functions)-1)]

            if node.data == 0:
                node.data = 'x'
            elif callable(node.data):  
                node.left = geneNode()
                node.right = geneNode()
                insertRandomNode(node.left, depth=depth+1)
                insertRandomNode(node.right, depth=depth+1)

        insertRandomNode(self, depth=d)

    def crossOver(self, tree2):
        pass


    def mutate(self):
        
        def mutateStep(node, depth=1):
            if random.random() < .1:
                newTree = geneNode()
                newTree.buildRandomTree(depth)
                node.left = newTree

            elif random.random() < .1:
                newTree = geneNode()
                newTree.buildRandomTree(depth)
                node.right = newTree
            else:
                if callable(node.left):
                    mutateStep(node.left, depth=depth+1)
                if callable(node.right):
                    mutateStep(node.right, depth=depth+1)

        mutateStep(self)


    def printTree(self, prefix="|-"):
        if callable(self.data):
            print(prefix, self.data.__name__)
        else:
            print(prefix, self.data)
        if self.left : self.left.printTree(prefix="     "+prefix)
        if self.right : self.right.printTree(prefix="     "+prefix)

    def calculateTree(self, x):
        if callable(self.data):
            return self.data(self.left.calculateTree(x), self.right.calculateTree(x))
        elif self.data == 'x':
            return x
        else: 
            return self.data

# Test

In [10]:
def add(x,y): return x+y
def sub(x,y): return x-y
def mult(x,y): return x*y

In [11]:
treeRoot = geneNode(maxDepth=5, functions=[add, sub, mult])
treeRoot.buildRandomTree()
treeRoot.printTree()
treeRoot.mutate()
treeRoot.printTree()

|- mult
     |- x
     |- mult
          |- add
               |- sub
                    |- sub
                         |- x
                         |- 1
                    |- x
               |- 1
          |- x
|- mult
     |- x
     |- 1


# Initialize Population

In [ ]:
def initializePopulation(popSize, func=[]):
    
    population = []

    for i in range(popSize):
    
        treeRoot = geneNode(maxDepth=5, functions=func)
        treeRoot.buildRandomTree()
        population.append(treeRoot)

    return population

# Evaluation and Selection

In [ ]:
def fitnessFunction(x):
    return x*x*x*x + x*x*x + x*x + x + 1
    

In [ ]:
#Roulette Wheel Selection (others include Rank method and Tournament Size)
def evaluatePopulation(population, point, testValues):
    
    populationFitness = []
    for individual in population:
        
        trialValues = []
        for p in point:
            trialValues.append(individual.calculateTree(p))
        
        fitness = sum([ (trialValues[i] - testValues[i]) for i in range(len(testValues)) ])
        populationFitness.append(fitness)

    return populationFitness


In [ ]:
def selection():
    pass

# Main

In [ ]:
generations = 200
popSize = 5

In [ ]:
def add(x,y): return x+y
def sub(x,y): return x-y
def mult(x,y): return x*y

In [ ]:
#INITIALIZE population
population = initializePopulation(popSize, func=[add, sub, mult])

#EVALUATE initial poplulation
trainPoints = range(100)
trueValues = []
for i in trainPoints:
    trueValues.append(fitnessFunction(i))

populationFitness = evaluatePopulation(population, trainPoints, trueValues)

globalBestFitness = max(populationFitness)
globalBestSolution = population[populationFitness.index(globalBestFitness)]

#Begin Evolution Search
for generation in range(generations):

    # SELECT Parents
    sortFitness = np.argsort(populationFitness)
    sortedPopulation = [population[i] for i in sortFitness]

    parent1 = population[1]
    parent2 = population[2]
    offspring = parent1.copy()

    newPopulation = []
    for i in range(len(population)):

        #RECOMBINE Parents
        if random.random() <= 0.87:
            offspring = crossover(parent1, parent2)

        #MUTATE Offspring
        if random.random() <= 0.7:
            offspring = mutation(offspring)

        newPopulation.append(offspring)
    
    #Evaluate newPopulation
    populationFitness = evaluatePopulation(newPopulation)

    #SELECT Survival
    population = newPopulation

|- mult
     |- mult
          |- 1
          |- add
               |- x
               |- 1
     |- sub
          |- sub
               |- -1
               |- mult
                    |- mult
                         |- -1
                         |- x
                    |- 1
          |- sub
               |- 1
               |- 1


In [ ]:
# print(globalBestSolution)